In [1]:
datos=spark.read.format("libsvm")\
.load("/usr/local/spark/data/mllib/sample_linear_regression_data.txt")

In [3]:
datos.show(1,truncate=False)

+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label             |features                                                                                                                                                                                                                            |
+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548|(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715])|


In [4]:
datos.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
lr=LinearRegression(featuresCol="features",
                    labelCol="label")

In [7]:
modelo=lr.fit(datos)

In [9]:
modelo.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
modelo.intercept

0.14228558260358093

In [12]:
#Coeficiente de determinación r2: 1 Bien, 0 Mal
modelo.summary.r2

0.027839179518600154

In [13]:
#Ahora con entrenamiento y evaluación

In [14]:
datos.count()

501

In [18]:
entrena,evalua=datos.randomSplit([0.7,0.3])

In [19]:
entrena.describe().show(),evalua.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 346|
|   mean|-0.09724383890000558|
| stddev|  10.504645143043856|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                155|
|   mean| 1.0474043108328555|
| stddev|  9.875589840735573|
|    min|-28.046018037776633|
|    max| 24.290551295953957|
+-------+-------------------+



(None, None)

In [20]:
modelo2=lr.fit(entrena)

In [21]:
evaluacion=modelo2.evaluate(evalua)

In [23]:
evaluacion.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...| -1.2236200028018565|
|-23.487440120936512|(10,[0,1,2,3,4,5,...| -1.7581699477277097|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|   0.979211163006701|
|-19.872991038068406|(10,[0,1,2,3,4,5,...| -0.1447926452646905|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|   2.224598345432747|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|  1.3750316571937415|
|-15.951512565794573|(10,[0,1,2,3,4,5,...| -1.9839965556482944|
| -13.15333560636553|(10,[0,1,2,3,4,5,...| -1.5031774538337692|
|-12.977848725392104|(10,[0,1,2,3,4,5,...| -0.6019775837251898|
| -12.92222310337042|(10,[0,1,2,3,4,5,...|  3.1316187102734383|
|-12.773226999251197|(10,[0,1,2,3,4,5,...|  -4.087889964404522|
|-12.491442077546413|(10,[0,1,2,3,4,5,...|  2.1137196591930745|
|-12.479280211451497|(10,[0,1,2,3,4,5,..

In [24]:
evaluacion.r2

-0.030029452441678428

In [25]:
evaluacion.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -26.82239803497478|
|-21.729270173208803|
|-20.863771937280127|
|-19.728198392803716|
|-19.653272916372252|
|-18.440431283069756|
|-13.967516010146278|
| -11.65015815253176|
|-12.375871141666913|
| -16.05384181364386|
| -8.685337034846675|
|-14.605161736739488|
|-11.861699257330601|
| -11.10488307244843|
|-10.375707872180563|
|-12.252798127404233|
|-12.699694173460344|
|-10.329808688999886|
| -7.985477601344364|
| -10.57136338011486|
+-------------------+
only showing top 20 rows



In [27]:
evaluacion.rootMeanSquaredError,evaluacion.meanSquaredError,
evaluacion.meanAbsoluteError,

(9.990388498830326, 99.80786235756123, 7.989915213687065)

In [28]:
datosNuevos=evalua.drop("label")#para simular datos nuevos

In [29]:
datosNuevos.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [30]:
pred=modelo2.transform(datosNuevos)

In [31]:
pred.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.2236200028018565|
|(10,[0,1,2,3,4,5,...| -1.7581699477277097|
|(10,[0,1,2,3,4,5,...|   0.979211163006701|
|(10,[0,1,2,3,4,5,...| -0.1447926452646905|
|(10,[0,1,2,3,4,5,...|   2.224598345432747|
|(10,[0,1,2,3,4,5,...|  1.3750316571937415|
|(10,[0,1,2,3,4,5,...| -1.9839965556482944|
|(10,[0,1,2,3,4,5,...| -1.5031774538337692|
|(10,[0,1,2,3,4,5,...| -0.6019775837251898|
|(10,[0,1,2,3,4,5,...|  3.1316187102734383|
|(10,[0,1,2,3,4,5,...|  -4.087889964404522|
|(10,[0,1,2,3,4,5,...|  2.1137196591930745|
|(10,[0,1,2,3,4,5,...| -0.6175809541208966|
|(10,[0,1,2,3,4,5,...|  -1.362773308584429|
|(10,[0,1,2,3,4,5,...|  -1.481642493248862|
|(10,[0,1,2,3,4,5,...|   0.637022862388606|
|(10,[0,1,2,3,4,5,...|  2.1163828735922143|
|(10,[0,1,2,3,4,5,...|-0.24788961524333508|
|(10,[0,1,2,3,4,5,...| -1.8229091012202931|
|(10,[0,1,2,3,4,5,...|  0.923638